In [1]:
import pandas as pd
from piex import explorer

In [2]:
piex = explorer.PipelineExplorer('ml-pipelines-2018')

In [3]:
df = piex.get_test_results(data_modality='single_table', task_type='classification')

In [4]:
def get_scores(df):
    stats = df.groupby('dataset')['rank'].agg(['mean', 'std', 'min', 'max'])
    stats['range'] = stats['max'] - stats['min']
    stats['midpoint'] = stats['min'] + stats['range'] / 2
    
    df = df.merge(stats, how='left', on='dataset')
    df['t-score'] = (df['rank'] - df['mean']) / df['std']
    df['r-score'] = (df['rank'] - df['midpoint']) / df['range']
    df['ranking'] = df.groupby('dataset')['rank'].rank(method='dense')
    
    return df

scores = get_scores(df)

In [5]:
def rank_pipelines(df, metric='t-score'):
    scores = get_scores(df)
    pipeline_scores = scores.groupby('pipeline')[['t-score', 'r-score', 'ranking']].mean()
    return pipeline_scores.sort_values(metric)

In [6]:
rank_pipelines(df)

,t-score,r-score,ranking
pipeline,,,
categorical_encoder/imputer/standard_scaler/xgbclassifier,-0.623300,-0.323580,3.067380
dfs/categorical_encoder/imputer/standard_scaler/xgbclassifier,-0.317369,-0.229551,4.577088
categorical_encoder/imputer/standard_scaler/random_forest_classifier,0.131168,-0.045852,5.897436
dfs/categorical_encoder/imputer/standard_scaler/random_forest_classifier,0.808199,0.179472,7.525641


In [7]:
def split_datasets(df):
    datasets = pd.Series(df.dataset.unique())
    datasets = datasets.sample(len(datasets))
    half = int(len(datasets) / 2)
    
    first_half = df[df.dataset.isin(datasets[:half])]
    second_half = df[df.dataset.isin(datasets[half:])]
    
    return first_half, second_half

first_half, second_half = split_datasets(df)

In [8]:
rank_pipelines(first_half)

,t-score,r-score,ranking
pipeline,,,
categorical_encoder/imputer/standard_scaler/xgbclassifier,-0.625437,-0.320651,3.042735
dfs/categorical_encoder/imputer/standard_scaler/xgbclassifier,-0.288555,-0.217782,4.672377
categorical_encoder/imputer/standard_scaler/random_forest_classifier,0.103277,-0.058294,5.850427
dfs/categorical_encoder/imputer/standard_scaler/random_forest_classifier,0.810076,0.175557,7.542735


In [9]:
rank_pipelines(second_half)

,t-score,r-score,ranking
pipeline,,,
categorical_encoder/imputer/standard_scaler/xgbclassifier,-0.621158,-0.326516,3.092077
dfs/categorical_encoder/imputer/standard_scaler/xgbclassifier,-0.346183,-0.241319,4.481799
categorical_encoder/imputer/standard_scaler/random_forest_classifier,0.159058,-0.033411,5.944444
dfs/categorical_encoder/imputer/standard_scaler/random_forest_classifier,0.806322,0.183387,7.508547
